# Step 1: Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn
import tensorflow
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam, Nadam
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2


#for data preprocessing
from google.colab import drive

#for loading the model
from tensorflow.keras.models import load_model



# Step 2: Data preperation



1.   Connecting google drive




In [2]:

drive.mount('/content/drive')


Mounted at /content/drive


2. Load the data into a pandas dataframe

In [3]:
path = '/content/drive/MyDrive/Artifical Minds - CMPS261/HIGGS_train.csv'
data = pd.read_csv(path)

data.info()

<ipython-input-3-7fae3bc4baac>:2: DtypeWarning: Columns (8,21) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 599999 entries, 0 to 599998
Data columns (total 29 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   1.00E+00    599999 non-null  float64
 1   8.69E-01    599999 non-null  float64
 2   -6.35E-01   599999 non-null  float64
 3   2.26E-01    599999 non-null  float64
 4   3.27E-01    599999 non-null  float64
 5   -6.90E-01   599999 non-null  float64
 6   7.54E-01    599999 non-null  float64
 7   -2.49E-01   599999 non-null  float64
 8   -1.09E+00   599999 non-null  object 
 9   0.00E+00    599999 non-null  float64
 10  1.37E+00    599999 non-null  float64
 11  -6.54E-01   599999 non-null  float64
 12  9.30E-01    599999 non-null  float64
 13  1.11E+00    599999 non-null  float64
 14  1.14E+00    599999 non-null  float64
 15  -1.58E+00   599999 non-null  float64
 16  -1.05E+00   599999 non-null  float64
 17  0.00E+00.1  599998 non-null  float64
 18  6.58E-01    599999 non-null  float64
 19  -1

3. Naming the columns

In [4]:
column_names = ['class_label', 'lepton_pt', 'lepton_eta', 'lepton_phi', 'missing_energy_magnitude', 'missing_energy_phi',
                'jet_1_pt', 'jet_1_eta', 'jet_1_phi', 'jet_1_b-tag', 'jet_2_pt', 'jet_2_eta', 'jet_2_phi', 'jet_2_b-tag',
                'jet_3_pt', 'jet_3_eta', 'jet_3_phi', 'jet_3_b-tag', 'jet_4_pt', 'jet_4_eta', 'jet_4_phi', 'jet_4_b-tag',
                'm_jj', 'm_jjj', 'm_lv', 'm_jlv', 'm_bb', 'm_wbb', 'm_wwbb']
data.columns = column_names

4. Checking for null values

In [5]:
print(data.isna().sum())
data['jet_1_phi'] = pd.to_numeric(data['jet_1_phi'], errors='coerce')
data['jet_4_b-tag'] = pd.to_numeric(data['jet_4_b-tag'], errors='coerce')
data['class_label'] = data['class_label'].astype(int)

data = data.dropna();
data.drop_duplicates(inplace=True)
data.info()

class_label                 0
lepton_pt                   0
lepton_eta                  0
lepton_phi                  0
missing_energy_magnitude    0
missing_energy_phi          0
jet_1_pt                    0
jet_1_eta                   0
jet_1_phi                   0
jet_1_b-tag                 0
jet_2_pt                    0
jet_2_eta                   0
jet_2_phi                   0
jet_2_b-tag                 0
jet_3_pt                    0
jet_3_eta                   0
jet_3_phi                   0
jet_3_b-tag                 1
jet_4_pt                    0
jet_4_eta                   0
jet_4_phi                   0
jet_4_b-tag                 0
m_jj                        0
m_jjj                       0
m_lv                        0
m_jlv                       0
m_bb                        0
m_wbb                       0
m_wwbb                      0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
Int64Index: 599193 entries, 0 to 599998
Data columns (total 29 columns):
 #   C

# Step 3: Data Scaling

1.  Split the data into features and labels


In [6]:
X = data.iloc[:, 1:]
y = data.iloc[:, 0]
print(y.values)

[1 1 0 ... 1 1 0]


2. Feature scaling

In [8]:
scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

#Step 4: Model architecture 

In [10]:
def create_improved_model(activation='relu', optimizer='adam'):
    inputs = Input(shape=(X.shape[1],))
    x = Dense(1024, activation=activation)(inputs)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(512, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.4)(x)
    x = Dense(256, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Dense(128, activation=activation)(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=inputs, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# Create the model
improved_model = KerasClassifier(build_fn=create_improved_model, epochs=50, batch_size=256 , verbose=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Add EarlyStopping and ReduceLROnPlateau callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=0.00001)

# Train and evaluate the model
history = improved_model.fit(X_train, y_train, validation_split=0.33, callbacks=[early_stopping, lr_reduction], verbose=1)
test_accuracy = improved_model.score(X_test, y_test)
print("Test accuracy:", test_accuracy)


<ipython-input-10-d26db877cb6e>:22: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  improved_model = KerasClassifier(build_fn=create_improved_model, epochs=50, batch_size=256 , verbose=1)


Epoch 1/50
1255/1255 [==============================] - 54s 42ms/step - loss: 0.6216 - accuracy: 0.6588 - val_loss: 0.5574 - val_accuracy: 0.7086 - lr: 0.0010
Epoch 2/50
1255/1255 [==============================] - 52s 42ms/step - loss: 0.5655 - accuracy: 0.7052 - val_loss: 0.5370 - val_accuracy: 0.7252 - lr: 0.0010
Epoch 3/50
1255/1255 [==============================] - 53s 42ms/step - loss: 0.5515 - accuracy: 0.7154 - val_loss: 0.5304 - val_accuracy: 0.7290 - lr: 0.0010
Epoch 4/50
1255/1255 [==============================] - 52s 42ms/step - loss: 0.5436 - accuracy: 0.7211 - val_loss: 0.5240 - val_accuracy: 0.7330 - lr: 0.0010
Epoch 5/50
1255/1255 [==============================] - 52s 42ms/step - loss: 0.5383 - accuracy: 0.7245 - val_loss: 0.5217 - val_accuracy: 0.7352 - lr: 0.0010
Epoch 6/50
1255/1255 [==============================] - 52s 42ms/step - loss: 0.5333 - accuracy: 0.7284 - val_loss: 0.5162 - val_accuracy: 0.7394 - lr: 0.0010
Epoch 7/50
1255/1255 [========================

#Step 5: Saving, loading and printing the info of our model



In [ ]:
# Save the model
improved_model.model.save('my_model.h5')

In [16]:

# Load the saved model
loaded_model = load_model('my_model.h5')

# Print the summary of the loaded model
loaded_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28)]              0         
                                                                 
 dense_5 (Dense)             (None, 1024)              29696     
                                                                 
 batch_normalization_4 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 1024)              0         
                                                                 
 dense_6 (Dense)             (None, 512)               524800    
                                                                 
 batch_normalization_5 (Batc  (None, 512)              2048      
 hNormalization)                                           